In [23]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [24]:
import zipfile
from pathlib import Path

###  !rm -rf <folder_name>

data_path = Path("data/")
image_path = data_path

with zipfile.ZipFile("drive/MyDrive/emp/case1.zip", "r") as zip_ref:
  print("Unzipping case1...")
  zip_ref.extractall(image_path)


Unzipping case1...


In [25]:
# # For this notebook to run with updated APIs, we need torch 1.12+ and torchvision 0.13+
# try:
#     import torch
#     import torchvision
#     assert int(torch.__version__.split(".")[1]) >= 12, "torch version should be 1.12+"
#     assert int(torchvision.__version__.split(".")[1]) >= 13, "torchvision version should be 0.13+"
#     print(f"torch version: {torch.__version__}")
#     print(f"torchvision version: {torchvision.__version__}")
# except:
#     print(f"[INFO] torch/torchvision versions not as required, installing nightly versions.")
#     !pip3 install -U torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu113
#     import torch
#     import torchvision
#     print(f"torch version: {torch.__version__}")
#     print(f"torchvision version: {torchvision.__version__}")

In [26]:
# Continue with regular imports
import matplotlib.pyplot as plt
import torch
import torchvision

from torch import nn
from torchvision import transforms

# Try to get torchinfo, install it if it doesn't work
try:
    from torchinfo import summary
except:
    print("[INFO] Couldn't find torchinfo... installing it.")
    !pip install -q torchinfo
    from torchinfo import summary

# Try to import the going_modular directory, download it from GitHub if it doesn't work
try:
    from going_modular.going_modular import data_setup, engine
    from helper_functions import download_data, set_seeds, plot_loss_curves
except:
    # Get the going_modular scripts
    print("[INFO] Couldn't find going_modular or helper_functions scripts... downloading them from GitHub.")
    !git clone https://github.com/mrdbourke/pytorch-deep-learning
    !mv pytorch-deep-learning/going_modular .
    !mv pytorch-deep-learning/helper_functions.py . # get the helper_functions.py script
    !rm -rf pytorch-deep-learning
    from going_modular.going_modular import data_setup, engine
    from helper_functions import download_data, set_seeds, plot_loss_curves

In [27]:
# Setup device agnostic code
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [28]:
# Setup Dirs
# train_dir = image_path / "emp/case/train"
# test_dir = image_path / "emp/case/test"

# train_dir , test_dir

In [29]:
# Setup dirs
train_dir = image_path / "case1/train"
test_dir = image_path / "case1/test"

# # Setup pretrained weights (plenty of these available in torchvision.models)
# weights = torchvision.models.EfficientNet_B0_Weights.DEFAULT

# # Get transforms from weights (these are the transforms that were used to obtain the weights)
# automatic_transforms = weights.transforms()
# print(f"Automatically created transforms: {automatic_transforms}")

# # Create data loaders
# train_dataloader, test_dataloader, class_names = data_setup.create_dataloaders(
#     train_dir=train_dir,
#     test_dir=test_dir,
#     transform=automatic_transforms, # use automatic created transforms
#     batch_size=32
# )

# train_dataloader, test_dataloader, class_names

In [30]:
# try:
#     from torch.utils.tensorboard import SummaryWriter
# except:
#     print("[INFO] Couldn't find tensorboard... installing it.")
#     !pip install -q tensorboard
#     from torch.utils.tensorboard import SummaryWriter


# # Create a writer with all default settings
# writer = SummaryWriter()

In [31]:
# def create_writer(experiment_name: str,
#                   model_name: str,
#                   extra: str=None) -> torch.utils.tensorboard.writer.SummaryWriter():
#     """Creates a torch.utils.tensorboard.writer.SummaryWriter() instance saving to a specific log_dir.

#     log_dir is a combination of runs/timestamp/experiment_name/model_name/extra.

#     Where timestamp is the current date in YYYY-MM-DD format.

#     Args:
#         experiment_name (str): Name of experiment.
#         model_name (str): Name of model.
#         extra (str, optional): Anything extra to add to the directory. Defaults to None.

#     Returns:
#         torch.utils.tensorboard.writer.SummaryWriter(): Instance of a writer saving to log_dir.

#     Example usage:
#         # Create a writer saving to "runs/2022-06-04/data_10_percent/effnetb2/5_epochs/"
#         writer = create_writer(experiment_name="data_10_percent",
#                                model_name="effnetb2",
#                                extra="5_epochs")
#         # The above is the same as:
#         writer = SummaryWriter(log_dir="runs/2022-06-04/data_10_percent/effnetb2/5_epochs/")
#     """
#     from datetime import datetime
#     import os

#     # Get timestamp of current date (all experiments on certain day live in same folder)
#     timestamp = datetime.now().strftime("%Y-%m-%d") # returns current date in YYYY-MM-DD format

#     if extra:
#         # Create log directory path
#         log_dir = os.path.join("runs", timestamp, str(experiment_name), model_name, extra)
#     else:
#         log_dir = os.path.join("runs", timestamp, str(experiment_name), model_name)

#     print(f"[INFO] Created SummaryWriter, saving to: {log_dir}...")
#     return SummaryWriter(log_dir=log_dir)

In [32]:
from typing import Dict, List
from tqdm.auto import tqdm

from going_modular.going_modular.engine import train_step, test_step

# Import train() function from:
# https://github.com/mrdbourke/pytorch-deep-learning/blob/main/going_modular/going_modular/engine.py
from typing import Dict, List
from tqdm.auto import tqdm
import wandb


# Add writer parameter to train()
def train(model: torch.nn.Module,
          train_dataloader: torch.utils.data.DataLoader,
          test_dataloader: torch.utils.data.DataLoader,
          optimizer: torch.optim.Optimizer,
          loss_fn: torch.nn.Module,
          epochs: int,
          device: torch.device,
          # writer: torch.utils.tensorboard.writer.SummaryWriter # new parameter to take in a writer
          ) -> Dict[str, List]:
    """Trains and tests a PyTorch model.

    Passes a target PyTorch models through train_step() and test_step()
    functions for a number of epochs, training and testing the model
    in the same epoch loop.

    Calculates, prints and stores evaluation metrics throughout.

    Stores metrics to specified writer log_dir if present.

    Args:
      model: A PyTorch model to be trained and tested.
      train_dataloader: A DataLoader instance for the model to be trained on.
      test_dataloader: A DataLoader instance for the model to be tested on.
      optimizer: A PyTorch optimizer to help minimize the loss function.
      loss_fn: A PyTorch loss function to calculate loss on both datasets.
      epochs: An integer indicating how many epochs to train for.
      device: A target device to compute on (e.g. "cuda" or "cpu").
      writer: A SummaryWriter() instance to log model results to.

    Returns:
      A dictionary of training and testing loss as well as training and
      testing accuracy metrics. Each metric has a value in a list for
      each epoch.
      In the form: {train_loss: [...],
                train_acc: [...],
                test_loss: [...],
                test_acc: [...]}
      For example if training for epochs=2:
              {train_loss: [2.0616, 1.0537],
                train_acc: [0.3945, 0.3945],
                test_loss: [1.2641, 1.5706],
                test_acc: [0.3400, 0.2973]}
    """
    # Create empty results dictionary
    results = {"train_loss": [],
               "train_acc": [],
               "test_loss": [],
               "test_acc": []
    }

    # Loop through training and testing steps for a number of epochs
    for epoch in tqdm(range(epochs)):
        train_loss, train_acc = train_step(model=model,
                                          dataloader=train_dataloader,
                                          loss_fn=loss_fn,
                                          optimizer=optimizer,
                                          device=device)
        test_loss, test_acc = test_step(model=model,
                                        dataloader=test_dataloader,
                                        loss_fn=loss_fn,
                                        device=device)

        # Print out what's happening
        print(
          f"Epoch: {epoch+1} | "
          f"train_loss: {train_loss:.4f} | "
          f"train_acc: {train_acc:.4f} | "
          f"test_loss: {test_loss:.4f} | "
          f"test_acc: {test_acc:.4f}"
        )

        # Update results dictionary
        results["train_loss"].append(train_loss)
        results["train_acc"].append(train_acc)
        results["test_loss"].append(test_loss)
        results["test_acc"].append(test_acc)


        ### New: Use the writer parameter to track experiments ###
        # See if there's a writer, if so, log to it
        # if writer:
        #     # Add results to SummaryWriter
        #     writer.add_scalars(main_tag="Loss",
        #                        tag_scalar_dict={"train_loss": train_loss,
        #                                         "test_loss": test_loss},
        #                        global_step=epoch)
        #     writer.add_scalars(main_tag="Accuracy",
        #                        tag_scalar_dict={"train_acc": train_acc,
        #                                         "test_acc": test_acc},
        #                        global_step=epoch)

        #     # Close the writer
        #     writer.close()
        # else:
        #     pass
        ### End new ###

        # Log metrics to wandb
        wandb.log({
            "train_loss": train_loss,
            "train_acc": train_acc,
            "test_loss": test_loss,
            "test_acc": test_acc,
            "epoch": epoch + 1
        })


    # Return the filled results at the end of the epochs
    return results

In [33]:
# Setup pretrained weights (plenty of these available in torchvision.models)
effnetv2_weights = torchvision.models.EfficientNet_V2_M_Weights.DEFAULT
inception_weights = torchvision.models.Inception_V3_Weights.DEFAULT
mobilenetv3_weights = torchvision.models.MobileNet_V3_Large_Weights.DEFAULT
resnet50_weights = torchvision.models.ResNet50_Weights.DEFAULT
vgg16bn_weights =  torchvision.models.VGG16_BN_Weights.DEFAULT



# Get transforms from weights (these are the transforms that were used to obtain the weights)
effnet_transforms = effnetv2_weights.transforms()
print(f"Automatically created transforms: {effnet_transforms}")
inception_transforms = inception_weights.transforms()
print(f"Automatically created transforms: {inception_transforms}")
mobilenet_transforms = mobilenetv3_weights.transforms()
print(f"Automatically created transforms: {mobilenet_transforms}")
resnet50_transforms = resnet50_weights.transforms()
print(f"Automatically created transforms: {resnet50_transforms}")
vgg16bn_transforms = vgg16bn_weights.transforms()
print(f"Automatically created transforms: {vgg16bn_transforms}")


Automatically created transforms: ImageClassification(
    crop_size=[480]
    resize_size=[480]
    mean=[0.485, 0.456, 0.406]
    std=[0.229, 0.224, 0.225]
    interpolation=InterpolationMode.BILINEAR
)
Automatically created transforms: ImageClassification(
    crop_size=[299]
    resize_size=[342]
    mean=[0.485, 0.456, 0.406]
    std=[0.229, 0.224, 0.225]
    interpolation=InterpolationMode.BILINEAR
)
Automatically created transforms: ImageClassification(
    crop_size=[224]
    resize_size=[232]
    mean=[0.485, 0.456, 0.406]
    std=[0.229, 0.224, 0.225]
    interpolation=InterpolationMode.BILINEAR
)
Automatically created transforms: ImageClassification(
    crop_size=[224]
    resize_size=[232]
    mean=[0.485, 0.456, 0.406]
    std=[0.229, 0.224, 0.225]
    interpolation=InterpolationMode.BILINEAR
)
Automatically created transforms: ImageClassification(
    crop_size=[224]
    resize_size=[256]
    mean=[0.485, 0.456, 0.406]
    std=[0.229, 0.224, 0.225]
    interpolation=Inte

In [34]:
BATCH_SIZE = 32

# Create efficientnet_v2_m training and test DataLoaders
train_dataloader_effnet, test_dataloader_effnet, class_names = data_setup.create_dataloaders(train_dir=train_dir,
                                                                                             test_dir=test_dir,
                                                                                              transform=effnet_transforms,
                                                                                                batch_size=BATCH_SIZE
                                                                                                                      )

# Create inception_v3 training and test data DataLoders
train_dataloader_inception, test_dataloader_inception, class_names = data_setup.create_dataloaders(train_dir=train_dir,
    test_dir=test_dir,
    transform=inception_transforms,
    batch_size=BATCH_SIZE
)
# Create mobilenet_v3_large training and test DataLoaders
train_dataloader_mobilenet, test_dataloader_mobilenet, class_names = data_setup.create_dataloaders(train_dir=train_dir,
    test_dir=test_dir,
    transform=mobilenet_transforms,
    batch_size=BATCH_SIZE
)
# Create resnet50 training and test DataLoaders
train_dataloader_resnet50, test_dataloader_resnet50, class_names = data_setup.create_dataloaders(train_dir=train_dir,
    test_dir=test_dir,
    transform=resnet50_transforms,
    batch_size=BATCH_SIZE
)
# Create vgg16_bn training and test DataLoaders
train_dataloader_vgg16, test_dataloader_vgg16, class_names = data_setup.create_dataloaders(train_dir=train_dir,
    test_dir=test_dir,
    transform=vgg16bn_transforms,
    batch_size=BATCH_SIZE
)

# Could increase if we had more samples, such as here: https://arxiv.org/abs/2205.01580 (there are other improvements there too...)

In [35]:
import torchvision
from torch import nn

# Get num out features (one for each class pizza, steak, sushi)
OUT_FEATURES = len(class_names)

# effnetv2_weights = torchvision.models.EfficientNet_V2_M_Weights.DEFAULT
# inception_weights = torchvision.models.Inception_V3_Weights.DEFAULT
# mobilenetv3_weights = torchvision.models.MobileNet_V3_Large_Weights.DEFAULT
# resnet50_weights = torchvision.models.ResNet50_Weights.DEFAULT
# vgg16bn_weights =  torchvision.models.VGG16_BN_Weights.DEFAULT

#1. Create an EffNetB0 feature extractor
def create_effnetv2():
    # 1. Get the base model with pretrained weights and send to target device
    effnetv2_weights = torchvision.models.EfficientNet_V2_M_Weights.DEFAULT
    model = torchvision.models.efficientnet_v2_m(weights=effnetv2_weights).to(device)

    # 2. Freeze the base model layers
    for param in model.features.parameters():
        param.requires_grad = False


    # 4. Change the classifier head
    model.classifier = nn.Sequential(
        nn.Dropout(p=0.3),
        nn.Linear(in_features=1280, out_features=OUT_FEATURES)
    ).to(device)

    # 5. Give the model a name
    model.name = "effnetv2"
    print(f"[INFO] Created new {model.name} model.")
    return model

#2. Create an inception feature extractor
def create_inception():
    # 1. Get the base model with pretrained weights and send to target device
    inception_weights = torchvision.models.Inception_V3_Weights.DEFAULT
    model = torchvision.models.inception_v3(weights=inception_weights).to(device)

    #model.aux_logits = False
    # 2. Freeze the base model layers
    for param in model.parameters():
        param.requires_grad = False


    # 4. Change the classifier head
    # Handle the auxilary net
    # model.AuxLogits.fc = nn.Linear(768, num_classes)
    model.aux_logits = False
    # Handle the primary net
    # model.fc = nn.Linear(in_features=2048, out_features=OUT_FEATURES, bias=True).to(device)
    model.fc = nn.Sequential(nn.Linear(model.fc.in_features, 10),
                             nn.Linear(10, 2)).to(device)

    # 5. Give the model a name
    model.name = "inception"
    print(f"[INFO] Created new {model.name} model.")
    # 6. Modify the forward method to return only the logits
    return model

#3. Create an mobilenetv3 feature extractor
def create_mobilenetv3():
    # 1. Get the base model with pretrained weights and send to target device
    mobilenetv3_weights = torchvision.models.MobileNet_V3_Large_Weights.DEFAULT
    model = torchvision.models.mobilenet_v3_large(weights=mobilenetv3_weights).to(device)

    # 2. Freeze the base model layers
    for param in model.parameters():
        param.requires_grad = False


    # 4. Change the classifier head
    model.classifier[3] = nn.Linear(in_features=1280, out_features=OUT_FEATURES).to(device)

    # 5. Give the model a name
    model.name = "mobilenetv3"
    print(f"[INFO] Created new {model.name} model.")
    return model

#4. Create an resnet50 feature extractor
def create_resnet50():
    # 1. Get the base model with pretrained weights and send to target device
    resnet50_weights = torchvision.models.ResNet50_Weights.DEFAULT
    model = torchvision.models.resnet50(weights=resnet50_weights).to(device)

    # 2. Freeze the base model layers
    for param in model.parameters():
        param.requires_grad = False


    # 4. Change the classifier head
    model.fc = nn.Linear(in_features=2048, out_features=OUT_FEATURES, bias=True).to(device)

    # 5. Give the model a name
    model.name = "resnet50"
    print(f"[INFO] Created new {model.name} model.")
    return model

#5. Create an vgg16bn feature extractor
def create_vgg16bn():
    # 1. Get the base model with pretrained weights and send to target device
    vgg16bn_weights =  torchvision.models.VGG16_BN_Weights.DEFAULT
    model = torchvision.models.vgg16_bn(weights=vgg16bn_weights).to(device)

    # 2. Freeze the base model layers
    for param in model.parameters():
        param.requires_grad = False


    # 4. Change the classifier head
    model.classifier[6] = nn.Linear(in_features=4096, out_features=OUT_FEATURES, bias=True).to(device)

    # 5. Give the model a name
    model.name = "vgg16bn"
    print(f"[INFO] Created new {model.name} model.")
    return model





In [36]:
effnetv2 = create_effnetv2()

# Get an output summary of the layers in our EffNetB0 feature extractor model (uncomment to view full output)
summary(model=effnetv2,
        input_size=(32, 3, 224, 224), # make sure this is "input_size", not "input_shape"
        # col_names=["input_size"], # uncomment for smaller output
        col_names=["input_size", "output_size", "num_params", "trainable"],
        col_width=20,
        row_settings=["var_names"]
)

[INFO] Created new effnetv2 model.


Layer (type (var_name))                                      Input Shape          Output Shape         Param #              Trainable
EfficientNet (EfficientNet)                                  [32, 3, 224, 224]    [32, 2]              --                   Partial
├─Sequential (features)                                      [32, 3, 224, 224]    [32, 1280, 7, 7]     --                   False
│    └─Conv2dNormActivation (0)                              [32, 3, 224, 224]    [32, 24, 112, 112]   --                   False
│    │    └─Conv2d (0)                                       [32, 3, 224, 224]    [32, 24, 112, 112]   (648)                False
│    │    └─BatchNorm2d (1)                                  [32, 24, 112, 112]   [32, 24, 112, 112]   (48)                 False
│    │    └─SiLU (2)                                         [32, 24, 112, 112]   [32, 24, 112, 112]   --                   --
│    └─Sequential (1)                                        [32, 24, 112, 112]   [32, 

In [37]:
inception = create_inception()

# Get an output summary of the layers in our EffNetB2 feature extractor model (uncomment to view full output)
summary(model=inception,
        input_size=(32, 3, 224, 224), # make sure this is "input_size", not "input_shape"
        # col_names=["input_size"], # uncomment for smaller output
        col_names=["input_size", "output_size", "num_params", "trainable"],
        col_width=20,
        row_settings=["var_names"]
)

[INFO] Created new inception model.


Layer (type (var_name))                  Input Shape          Output Shape         Param #              Trainable
Inception3 (Inception3)                  [32, 3, 224, 224]    [32, 2]              3,326,696            Partial
├─BasicConv2d (Conv2d_1a_3x3)            [32, 3, 224, 224]    [32, 32, 111, 111]   --                   False
│    └─Conv2d (conv)                     [32, 3, 224, 224]    [32, 32, 111, 111]   (864)                False
│    └─BatchNorm2d (bn)                  [32, 32, 111, 111]   [32, 32, 111, 111]   (64)                 False
├─BasicConv2d (Conv2d_2a_3x3)            [32, 32, 111, 111]   [32, 32, 109, 109]   --                   False
│    └─Conv2d (conv)                     [32, 32, 111, 111]   [32, 32, 109, 109]   (9,216)              False
│    └─BatchNorm2d (bn)                  [32, 32, 109, 109]   [32, 32, 109, 109]   (64)                 False
├─BasicConv2d (Conv2d_2b_3x3)            [32, 32, 109, 109]   [32, 64, 109, 109]   --                   False
│   

In [38]:
mobilenetv3 =  create_mobilenetv3()

# Get an output summary of the layers in our EffNetB2 feature extractor model (uncomment to view full output)
summary(model=mobilenetv3,
        input_size=(32, 3, 224, 224), # make sure this is "input_size", not "input_shape"
        # col_names=["input_size"], # uncomment for smaller output
        col_names=["input_size", "output_size", "num_params", "trainable"],
        col_width=20,
        row_settings=["var_names"]
)

[INFO] Created new mobilenetv3 model.


Layer (type (var_name))                                      Input Shape          Output Shape         Param #              Trainable
MobileNetV3 (MobileNetV3)                                    [32, 3, 224, 224]    [32, 2]              --                   Partial
├─Sequential (features)                                      [32, 3, 224, 224]    [32, 960, 7, 7]      --                   False
│    └─Conv2dNormActivation (0)                              [32, 3, 224, 224]    [32, 16, 112, 112]   --                   False
│    │    └─Conv2d (0)                                       [32, 3, 224, 224]    [32, 16, 112, 112]   (432)                False
│    │    └─BatchNorm2d (1)                                  [32, 16, 112, 112]   [32, 16, 112, 112]   (32)                 False
│    │    └─Hardswish (2)                                    [32, 16, 112, 112]   [32, 16, 112, 112]   --                   --
│    └─InvertedResidual (1)                                  [32, 16, 112, 112]   [32, 

In [39]:
resnet50 = create_resnet50()

# Get an output summary of the layers in our EffNetB2 feature extractor model (uncomment to view full output)
summary(model=resnet50,
        input_size=(32, 3, 224, 224), # make sure this is "input_size", not "input_shape"
        # col_names=["input_size"], # uncomment for smaller output
        col_names=["input_size", "output_size", "num_params", "trainable"],
        col_width=20,
        row_settings=["var_names"]
)

[INFO] Created new resnet50 model.


Layer (type (var_name))                  Input Shape          Output Shape         Param #              Trainable
ResNet (ResNet)                          [32, 3, 224, 224]    [32, 2]              --                   Partial
├─Conv2d (conv1)                         [32, 3, 224, 224]    [32, 64, 112, 112]   (9,408)              False
├─BatchNorm2d (bn1)                      [32, 64, 112, 112]   [32, 64, 112, 112]   (128)                False
├─ReLU (relu)                            [32, 64, 112, 112]   [32, 64, 112, 112]   --                   --
├─MaxPool2d (maxpool)                    [32, 64, 112, 112]   [32, 64, 56, 56]     --                   --
├─Sequential (layer1)                    [32, 64, 56, 56]     [32, 256, 56, 56]    --                   False
│    └─Bottleneck (0)                    [32, 64, 56, 56]     [32, 256, 56, 56]    --                   False
│    │    └─Conv2d (conv1)               [32, 64, 56, 56]     [32, 64, 56, 56]     (4,096)              False
│    │    

In [40]:
vgg16bn = create_vgg16bn()

# Get an output summary of the layers in our EffNetB2 feature extractor model (uncomment to view full output)
summary(model=vgg16bn,
        input_size=(32, 3, 224, 224), # make sure this is "input_size", not "input_shape"
        # col_names=["input_size"], # uncomment for smaller output
        col_names=["input_size", "output_size", "num_params", "trainable"],
        col_width=20,
        row_settings=["var_names"]
)

[INFO] Created new vgg16bn model.


Layer (type (var_name))                  Input Shape          Output Shape         Param #              Trainable
VGG (VGG)                                [32, 3, 224, 224]    [32, 2]              --                   Partial
├─Sequential (features)                  [32, 3, 224, 224]    [32, 512, 7, 7]      --                   False
│    └─Conv2d (0)                        [32, 3, 224, 224]    [32, 64, 224, 224]   (1,792)              False
│    └─BatchNorm2d (1)                   [32, 64, 224, 224]   [32, 64, 224, 224]   (128)                False
│    └─ReLU (2)                          [32, 64, 224, 224]   [32, 64, 224, 224]   --                   --
│    └─Conv2d (3)                        [32, 64, 224, 224]   [32, 64, 224, 224]   (36,928)             False
│    └─BatchNorm2d (4)                   [32, 64, 224, 224]   [32, 64, 224, 224]   (128)                False
│    └─ReLU (5)                          [32, 64, 224, 224]   [32, 64, 224, 224]   --                   --
│    └─Max

In [41]:
# 1. Create epochs list
num_epochs = 50

# 2. Create models list (need to create a new model for each experiment)
models = ["effnetv2","inception","mobilenetv3","resnet50","vgg16bn"]

# 3. Create dataloaders dictionary for various dataloaders
# train_dataloaders = {"data_train_effnet": train_dataloader_effnet,
#                      "data_train_inception": train_dataloader_inception,
#                      "data_train_mobilenet": train_dataloader_mobilenet,
#                      "data_train_resnet50": train_dataloader_resnet50,
#                      "data_train_vgg16": train_dataloader_vgg16}

In [42]:
import wandb
wandb.login()

True

In [44]:
%%time
from going_modular.going_modular.utils import save_model
import wandb

# 1. Set the random seeds
# set_seeds(seed=42)

# 2. Keep track of experiment numbers
experiment_number = 0

for model_name in models:

    # 6. Create information print outs
    experiment_number += 1
    print(f"[INFO] Experiment number: {experiment_number}")
    print(f"[INFO] Model: {model_name}")
    #print(f"[INFO] DataLoader: {dataloader_name}")
    print(f"[INFO] Number of epochs: {num_epochs}")

    # 7. Select the model
    if model_name == "effnetv2":
        model = create_effnetv2() # creates a new model each time (important because we want each experiment to start from scratch)
        train_dataloader=train_dataloader_effnet
        test_dataloader=test_dataloader_effnet

    elif model_name == "inception":
        model = create_inception() # creates a new model each time (important because we want each experiment to start from scratch)
        train_dataloader=train_dataloader_inception
        test_dataloader=test_dataloader_inception

    elif model_name == "mobilenetv3":
        model = create_mobilenetv3() # creates a new model each time (important because we want each experiment to start from scratch)
        train_dataloader=train_dataloader_mobilenet
        test_dataloader=test_dataloader_mobilenet

    elif model_name == "resnet50":
        model = create_resnet50() # creates a new model each time (important because we want each experiment to start from scratch)
        train_dataloader=train_dataloader_resnet50
        test_dataloader=test_dataloader_resnet50

    else:
        model = create_vgg16bn() # creates a new model each time (important because we want each experiment to start from scratch)
        train_dataloader=train_dataloader_vgg16
        test_dataloader=test_dataloader_vgg16



    # 8. Create a new loss and optimizer for every model
    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(params=model.parameters(), lr=0.001)

    # Initialize a new wandb run for each experiment
    wandb.init(project="your-project-name",
               name=f"{model_name}_{num_epochs}_epochs_exp_{experiment_number}",
               config={
                   "model_name": model_name,
                   "num_epochs": num_epochs,
                   "batch_size": BATCH_SIZE,
                   "learning_rate": 0.001,
                   "loss_fn": loss_fn.__class__.__name__,
                   "optimizer": optimizer.__class__.__name__
               })



    # 9. Train target model with target dataloaders and track experiments
    train(model=model,
          train_dataloader=train_dataloader,
          test_dataloader=test_dataloader,
          optimizer=optimizer,
          loss_fn=loss_fn,
          epochs=num_epochs,
          device=device,
          # writer=create_writer(experiment_name=experiment_number,
          #                       model_name=model_name,
          #                       extra=f"{num_epochs}_epochs")
          )

    # 10. Save the model to file so we can get back the best model
    save_filepath = f"01_{model_name}_{num_epochs}_epochs.pth"
    save_model(model=model,
                target_dir="models",
                model_name=save_filepath)
    print("-"*50 + "\n")

[INFO] Experiment number: 1
[INFO] Model: effnetv2
[INFO] Number of epochs: 50
[INFO] Created new effnetv2 model.


  0%|          | 0/50 [00:00<?, ?it/s]

Epoch: 1 | train_loss: 0.6050 | train_acc: 0.6706 | test_loss: 0.5087 | test_acc: 0.8281
Epoch: 2 | train_loss: 0.4409 | train_acc: 0.8743 | test_loss: 0.5105 | test_acc: 0.7266
Epoch: 3 | train_loss: 0.3470 | train_acc: 0.9121 | test_loss: 0.3018 | test_acc: 0.9375
Epoch: 4 | train_loss: 0.3001 | train_acc: 0.9251 | test_loss: 0.2529 | test_acc: 0.9453
Epoch: 5 | train_loss: 0.2661 | train_acc: 0.9368 | test_loss: 0.2292 | test_acc: 0.9531
Epoch: 6 | train_loss: 0.2231 | train_acc: 0.9629 | test_loss: 0.1862 | test_acc: 0.9531
Epoch: 7 | train_loss: 0.2257 | train_acc: 0.9492 | test_loss: 0.1644 | test_acc: 0.9531
Epoch: 8 | train_loss: 0.2132 | train_acc: 0.9427 | test_loss: 0.1718 | test_acc: 0.9609
Epoch: 9 | train_loss: 0.1753 | train_acc: 0.9648 | test_loss: 0.1514 | test_acc: 0.9531
Epoch: 10 | train_loss: 0.1829 | train_acc: 0.9486 | test_loss: 0.1557 | test_acc: 0.9531
Epoch: 11 | train_loss: 0.1801 | train_acc: 0.9427 | test_loss: 0.1334 | test_acc: 0.9609
Epoch: 12 | train_l

epoch,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▇▇▇▇▇▇███
test_acc,▄▁▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇█▇▇█
test_loss,██▅▄▄▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_acc,▁▆▆▇▇▇▇█▇▇▇▇▇█▇█▇█████████▇█████████████
train_loss,█▆▅▄▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▂▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch,50
test_acc,1
test_loss,0.04532
train_acc,0.97266
train_loss,0.08237


  0%|          | 0/50 [00:00<?, ?it/s]

...

  0%|          | 0/50 [00:00<?, ?it/s]

Epoch: 1 | train_loss: 0.6101 | train_acc: 0.7272 | test_loss: 0.6459 | test_acc: 0.5234
Epoch: 2 | train_loss: 0.4617 | train_acc: 0.9141 | test_loss: 1.0780 | test_acc: 0.4141
Epoch: 3 | train_loss: 0.3842 | train_acc: 0.8783 | test_loss: 0.5872 | test_acc: 0.5859
Epoch: 4 | train_loss: 0.3496 | train_acc: 0.9095 | test_loss: 0.5179 | test_acc: 0.7188
Epoch: 5 | train_loss: 0.2992 | train_acc: 0.9284 | test_loss: 0.3236 | test_acc: 0.9062
Epoch: 6 | train_loss: 0.2626 | train_acc: 0.9473 | test_loss: 0.2728 | test_acc: 0.9375
Epoch: 7 | train_loss: 0.2345 | train_acc: 0.9551 | test_loss: 0.2636 | test_acc: 0.9375
Epoch: 8 | train_loss: 0.2258 | train_acc: 0.9447 | test_loss: 0.2310 | test_acc: 0.9375
Epoch: 9 | train_loss: 0.2039 | train_acc: 0.9609 | test_loss: 0.2147 | test_acc: 0.9688
Epoch: 10 | train_loss: 0.1896 | train_acc: 0.9629 | test_loss: 0.2094 | test_acc: 0.9531
Epoch: 11 | train_loss: 0.1714 | train_acc: 0.9707 | test_loss: 0.1985 | test_acc: 0.9531
Epoch: 12 | train_l

epoch,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▇▇▇▇▇▇███
test_acc,▂▁▃▅▇▇▇█████████████████████████████████
test_loss,▅█▅▄▃▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_acc,▁▆▅▆▆▇▇▇▇▇▇▇████▇█▇████████▇█████▇██████
train_loss,█▆▅▅▄▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▁▂▁▁▁▁▁▂▂▁▁▁▁▁▁▁▁▁
epoch,50
test_acc,0.97656
test_loss,0.08609
train_acc,1
train_loss,0.04295


  0%|          | 0/50 [00:00<?, ?it/s]

Epoch: 1 | train_loss: 0.5057 | train_acc: 0.7747 | test_loss: 0.2922 | test_acc: 0.8984
Epoch: 2 | train_loss: 0.2857 | train_acc: 0.9102 | test_loss: 0.2462 | test_acc: 0.9062
Epoch: 3 | train_loss: 0.2356 | train_acc: 0.9115 | test_loss: 0.1972 | test_acc: 0.9375
Epoch: 4 | train_loss: 0.2203 | train_acc: 0.9180 | test_loss: 0.1781 | test_acc: 0.9297
Epoch: 5 | train_loss: 0.2357 | train_acc: 0.9043 | test_loss: 0.1633 | test_acc: 0.9375
Epoch: 6 | train_loss: 0.2343 | train_acc: 0.9010 | test_loss: 0.1546 | test_acc: 0.9453
Epoch: 7 | train_loss: 0.1601 | train_acc: 0.9486 | test_loss: 0.1494 | test_acc: 0.9453
Epoch: 8 | train_loss: 0.1332 | train_acc: 0.9544 | test_loss: 0.1477 | test_acc: 0.9375
Epoch: 9 | train_loss: 0.1486 | train_acc: 0.9486 | test_loss: 0.1346 | test_acc: 0.9688
Epoch: 10 | train_loss: 0.1185 | train_acc: 0.9609 | test_loss: 0.1330 | test_acc: 0.9609
Epoch: 11 | train_loss: 0.1483 | train_acc: 0.9277 | test_loss: 0.1274 | test_acc: 0.9531
Epoch: 12 | train_l

In [ ]:
# # Viewing TensorBoard in Jupyter and Google Colab Notebooks (uncomment to view full TensorBoard instance)
# %load_ext tensorboard
# %tensorboard --logdir runs

In [ ]:
# %cp -av models drive/MyDrive/emp/

'models/01_effnetv2_50_epochs.pth' -> 'drive/MyDrive/emp/models/01_effnetv2_50_epochs.pth'
'models/01_inception_50_epochs.pth' -> 'drive/MyDrive/emp/models/01_inception_50_epochs.pth'
'models/01_mobilenetv3_50_epochs.pth' -> 'drive/MyDrive/emp/models/01_mobilenetv3_50_epochs.pth'
'models/01_resnet50_50_epochs.pth' -> 'drive/MyDrive/emp/models/01_resnet50_50_epochs.pth'
'models/01_vgg16bn_50_epochs.pth' -> 'drive/MyDrive/emp/models/01_vgg16bn_50_epochs.pth'


In [ ]:
# # Setup the best model filepath
# best_model_path = "models/07_effnetb2_data_20_percent_10_epochs.pth"

# # Instantiate a new instance of EffNetB2 (to load the saved state_dict() to)
# best_model = create_effnetb2()

# # Load the saved best model state_dict()
# best_model.load_state_dict(torch.load(best_model_path))


In [ ]:
# # Check the model file size
# from pathlib import Path

# # Get the model size in bytes then convert to megabytes
# effnetb2_model_size = Path(best_model_path).stat().st_size // (1024*1024)
# print(f"EfficientNetB2 feature extractor model size: {effnetb2_model_size} MB")

In [ ]:
# # Import function to make predictions on images and plot them
# # See the function previously created in section: https://www.learnpytorch.io/06_pytorch_transfer_learning/#6-make-predictions-on-images-from-the-test-set
# from going_modular.going_modular.predictions import pred_and_plot_image

# # Get a random list of 3 images from 20% test set
import random
num_images_to_plot = 3
test_image_path_list = list(Path(data_20_percent_path / "test").glob("*/*.jpg")) # get all test image paths from 20% dataset
test_image_path_sample = random.sample(population=test_image_path_list,
                                       k=num_images_to_plot) # randomly select k number of images

# Iterate through random test image paths, make predictions on them and plot them
for image_path in test_image_path_sample:
    pred_and_plot_image(model=best_model,
                        image_path=image_path,
                        class_names=class_names,
                        image_size=(224, 224))

NameError: name 'data_20_percent_path' is not defined

In [ ]:
# Predict on custom image
pred_and_plot_image(model=model,
                    image_path=custom_image_path,
                    class_names=class_names)

NameError: name 'pred_and_plot_image' is not defined